In [1]:
using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
using Revise
using EasyHybrid
using Lux
using Optimisers
using WGLMakie
using Random
using LuxCore
using CSV, DataFrames
using EasyHybrid.MLUtils
using Statistics
using Plots
using JLD2

version = "v20251209"
results_dir = joinpath(@__DIR__, "eval");

# targets
targets = [:BD, :SOCconc, :CF, :SOCdensity];

# input
df = CSV.read(joinpath(@__DIR__, "data/lucas_preprocessed_v20251125.csv"), DataFrame; normalizenames=true)
println("all data: ", size(df))
train_df = df[df.time .== 2018, :]
println("train size: ", size(train_df))
test_df  = df[df.time .!= 2018, :]
println("test size: ", size(test_df))

# predictor
predictors = Symbol.(names(df))[18:end-6]; # CHECK EVERY TIME 
nf = length(predictors)

# scales
scalers = Dict(
    :SOCconc   => 0.151, # g/kg, log(x+1)*0.151
    :CF        => 0.263, # percent, log(x+1)*0.263
    :BD        => 0.529, # g/cm3, x*0.529
    :SOCdensity => 0.167, # kg/m3, log(x)*0.167
);

all data: (56117, 385)
train size: (16743, 385)
test size: (39374, 385)


## SiNN

In [2]:
testid = "03_hybridNN";

# mechanistic model
function SOCD_model(; SOCconc, CF, oBD, mBD)
    ϵ = 1e-7

    # invert transforms
    soct = (exp.(SOCconc ./ scalers[:SOCconc]) .- 1) ./ 1000
    soct = clamp.(soct, ϵ, Inf)
    
    cft = (exp.(CF ./ scalers[:CF]) .- 1) ./ 100
    cft = clamp.(cft, 0, 0.99)

    # compute BD safely
    som = 1.724f0 .* soct
    
    denom = som .* mBD .+ (1f0 .- som) .* oBD
    denom = clamp.(denom, ϵ, Inf)

    BD = (oBD .* mBD) ./ denom
    BD = clamp.(BD, ϵ, Inf)

    # SOCdensity
    SOCdensity = soct .* 1000 .* BD .* (1 .- cft)
    SOCdensity = clamp.(SOCdensity, 1, Inf)

    # scale
    SOCdensity = log.(SOCdensity) .* scalers[:SOCdensity]
    BD = BD .* scalers[:BD]

    return (; BD, SOCconc, CF, SOCdensity, oBD, mBD)
end

# param bounds
parameters = (
    SOCconc = (0.01f0, 0.0f0, 1.0f0),   # fraction
    CF      = (0.15f0, 0.0f0, 1.0f0),   # fraction,
    oBD     = (0.20f0, 0.05f0, 0.40f0),  # also NN learnt, g/cm3
    mBD     = (1.20f0, 0.75f0, 2.0f0),  # NN leanrt
)

# define param for hybrid model
neural_param_names = [:SOCconc, :CF, :mBD, :oBD];
# global_param_names = [:oBD]
forcing = Symbol[];
# targets = [:BD, :SOCconc, :SOCdensity, :CF]  # defined above  # SOCconc is both a param and a target


In [3]:
# read in best configuration
df_param = CSV.read(
    joinpath(results_dir, "$(testid)_hyperparams_$(version).csv"),
    DataFrame
)

sort!(df_param, :r2, rev = true)
first(df_param, 5)
best = df_param[1, :]

activation_map = Dict(
    "relu"       => relu,
    "swish"      => swish,
    "gelu_tanh"  => gelu   
)

best_config = (
    h   = parse.(Int, split(replace(best.h, "(" => "", ")" => ""), ", ")),
    bs  = best.bs,
    lr  = best.lr,
    act = activation_map[best.act]
)

(h = [256, 128, 64, 32], bs = 512, lr = 0.0005, act = NNlib.gelu_tanh)

In [4]:
hm = constructHybridModel(
    predictors,
    forcing,
    targets,
    SOCD_model,
    parameters,
    neural_param_names,
    [];
    hidden_layers = collect(best_config.h),
    activation = best_config.act,
    scale_nn_outputs = true,
    input_batchnorm = false,
    start_from_default = true
)

rlt = train(
    hm, train_df, ();
    nepochs = 200,
    batchsize = best_config.bs,
    opt = AdamW(best_config.lr),
    training_loss = :mse,
    loss_types = [:mse, :r2],
    shuffleobs = true,       
    file_name = "$(testid)_temporal.check.jld2", # history
    random_seed = 42,
    patience = 20,
    return_model = :best,
    show_progress = true,
    plotting = false,
    hybrid_name = "$(testid)_temporal.check"
)

ps, st = rlt.ps, rlt.st

(x_test, y_test) = prepare_data(hm, test_df)
ŷ_test, st_test = hm(x_test, ps, LuxCore.testmode(st))

for var in [:SOCdensity, :BD, :SOCconc, :CF, :oBD, :mBD]
    if hasproperty(ŷ_test, var)
        test_df[!, Symbol("pred_", var)] = getproperty(ŷ_test, var)
    end
end

CSV.write(joinpath(results_dir, "$(testid)_tc_$version.csv"), test_df)


┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /opt/julia/packages/ProgressMeter/N660J/src/ProgressMeter.jl:607
Training loss  12%|████▊                                 |  ETA: 0:11:00
              epoch : 25
            targets : BD       SOCconc  CF       SOCdensity  sum    
     training-start : 0.02513  0.01378  0.04427  0.01379     0.09696
            current : 0.00624  0.00484  0.00733  0.00364     0.02205
   validation-start : 0.02395  0.01414  0.04446  0.01407     0.09662
            current : 0.01742  0.00773  0.03702  0.00693     0.06911┌ Warning: Early stopping at epoch 26 with best validation loss wrt mse: 0.0583185223799935
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273

"/mnt/tupi/HybridModeling/EasyDensity.jl/eval/03_hybridNN_tc_v20251209.csv"

## MultiNN

In [5]:
testid = "02_multiNN";

# read in best configuration
df_param = CSV.read(
    joinpath(results_dir, "$(testid)_hyperparams_$(version).csv"),
    DataFrame
)

sort!(df_param, :r2, rev = true)
first(df_param, 5)
best = df_param[1, :]

activation_map = Dict(
    "relu"       => relu,
    "swish"      => swish,
    "gelu_tanh"  => gelu   
)

best_config = (
    h   = parse.(Int, split(replace(best.h, "(" => "", ")" => ""), ", ")),
    bs  = best.bs,
    lr  = best.lr,
    act = activation_map[best.act]
)

(h = [512, 256, 128, 64, 32, 16], bs = 512, lr = 0.001, act = NNlib.gelu_tanh)

In [6]:
nn = EasyHybrid.constructNNModel(
    predictors, targets;
    hidden_layers = collect(best_config.h),
    activation = best_config.act,
    scale_nn_outputs = true,
    input_batchnorm = false
)

rlt = train(
    nn, train_df, ();
    nepochs = 200,
    batchsize = best_config.bs,
    opt = AdamW(best_config.lr),
    training_loss = :mse,
    loss_types = [:mse, :r2],
    shuffleobs = true,
    file_name = "$(testid)_temporal.check.jld2",
    random_seed = 42,
    patience = 15,
    yscale = identity,
    agg = mean,
    return_model = :best,
    show_progress = false,
    plotting = false,
    hybrid_name = "$(testid)_temporal.check"
)

ps, st = rlt.ps, rlt.st

(x_test, y_test) = prepare_data(nn, test_df)
ŷ_test, st_test = nn(x_test, ps, LuxCore.testmode(st))

for var in [:SOCdensity, :BD, :SOCconc, :CF]
    if hasproperty(ŷ_test, var)
        test_df[!, Symbol("pred_", var)] = getproperty(ŷ_test, var)
    end
end

CSV.write(joinpath(results_dir, "$(testid)_tc_$version.csv"), test_df)

[ Info: Plotting disabled.
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 18 with best validation loss wrt mse: 0.01457891872245258
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 3 of 200 epochs with best validation loss wrt mse: 0.01457891872245258


"/mnt/tupi/HybridModeling/EasyDensity.jl/eval/02_multiNN_tc_v20251209.csv"

## UniNN


In [7]:
testid = "01_uniNN"

# read in best configuration
df_param = CSV.read(
    joinpath(results_dir, "$(testid)_hyperparams_$(version).csv"),
    DataFrame
)
df_param = df_param[df_param.target .== "SOCconc", :]
sort!(df_param, :r2, rev = true)
first(df_param, 5)
best = df_param[1, :]

activation_map = Dict(
    "relu"       => relu,
    "swish"      => swish,
    "gelu_tanh"  => gelu   
)

best_config = (
    h   = parse.(Int, split(replace(best.h, "(" => "", ")" => ""), ", ")),
    bs  = best.bs,
    lr  = best.lr,
    act = activation_map[best.act]
)

(h = [512, 256, 128, 64, 32], bs = 256, lr = 0.001, act = NNlib.gelu_tanh)

In [8]:
nn = EasyHybrid.constructNNModel(
    predictors, targets;
    hidden_layers = collect(best_config.h),
    activation = best_config.act,
    scale_nn_outputs = true,
    input_batchnorm = false
)

rlt = train(
    nn, train_df, ();
    nepochs = 200,
    batchsize = best_config.bs,
    opt = AdamW(best_config.lr),
    training_loss = :mse,
    loss_types = [:mse, :r2],
    shuffleobs = true,
    file_name = "$(testid)_temporal.check.jld2",
    random_seed = 42,
    patience = 15,
    yscale = identity,
    agg = mean,
    return_model = :best,
    show_progress = false,
    plotting = false,
    hybrid_name = "$(testid)_temporal.check"
)

ps, st = rlt.ps, rlt.st

(x_test, y_test) = prepare_data(nn, test_df)
ŷ_test, st_test = nn(x_test, ps, LuxCore.testmode(st))

for var in [:SOCdensity, :BD, :SOCconc, :CF]
    if hasproperty(ŷ_test, var)
        test_df[!, Symbol("pred_", var)] = getproperty(ŷ_test, var)
    end
end

CSV.write(joinpath(results_dir, "$(testid)_tc_$version.csv"), test_df)

[ Info: Plotting disabled.
[ Info: Check the saved output (.png, .mp4, .jld2) from training at: /mnt/tupi/HybridModeling/EasyDensity.jl/output_tmp
┌ Warning: Early stopping at epoch 20 with best validation loss wrt mse: 0.014501327819971316
└ @ EasyHybrid /opt/julia/packages/EasyHybrid/n8FOE/src/train.jl:273
[ Info: Returning best model from epoch 5 of 200 epochs with best validation loss wrt mse: 0.014501327819971316


"/mnt/tupi/HybridModeling/EasyDensity.jl/eval/01_uniNN_tc_v20251209.csv"